## Train/Build the Model

In [1]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('fivethirtyeight')

df = pd.read_csv('DC_Properties.csv')
df.shape
removable_features = ['SOURCE', 'CENSUS_TRACT', 'CENSUS_BLOCK', 'SQUARE', 'USECODE', 'Unnamed: 0', 'X', 'Y']
df = df.drop(removable_features, axis=1)
df.shape

df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
df['SALE_YR'] = df['SALEDATE'].dt.year; df['SALE_MONTH'] = df['SALEDATE'].dt.month
df = df.drop('SALEDATE', axis=1)
df = df.drop('GIS_LAST_MOD_DTTM', axis=1)
df = df[~pd.isnull(df['PRICE'])]
df = df.drop(df[df.PRICE>2000000].index)
df.shape

df = df.loc[:, df.isnull().sum() < 0.4*df.shape[0]]
df = df.drop('ASSESSMENT_SUBNBHD', axis=1)
df = df.dropna(how='any')
df = df.drop(['LATITUDE','LONGITUDE','ZIPCODE'], axis=1)

df = df[df.FIREPLACES<=11]
df = df[df.AYB>=1825]
df = df[df.EYB>=1900]
df = df[df.LANDAREA<=50000]

df = pd.concat([df, pd.get_dummies(df[['AC','QUALIFIED','WARD','QUADRANT']])], axis=1)
df = df.drop(['AC','QUALIFIED','WARD','QUADRANT'], axis=1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['HEAT']=le.fit_transform(df['HEAT'])
df['ASSESSMENT_NBHD']=le.fit_transform(df['ASSESSMENT_NBHD'])
df.shape

df["PRICE"] = np.log1p(df["PRICE"])
df = df.drop(df[df.PRICE<10].index)
y = df['PRICE']; X = df.drop('PRICE', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, y_train)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

## Pickle (export) the trained model

In [2]:
from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(forest_reg, 'regmodel.pkl') 

['regmodel.pkl']